In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
import pandas as pd
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
sales_train.head()

In [ ]:
sales_train.describe()

In [ ]:
sales_train.dtypes

In [ ]:
sales_train.isnull().sum()

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.dtypes

In [ ]:
test.isnull().sum()

In [ ]:
from datetime import datetime

In [ ]:
con=sales_train['date']
sales_train['date']=pd.to_datetime(sales_train['date'])
sales_train.set_index('date',inplace=True)
sales_train.index

**Exploratory Data Analysis

In [ ]:
item_series=sales_train.loc[:,('item_cnt_day')]
item_series.plot()

In [ ]:
sales_train.plot()

In [ ]:
sales_train[['shop_id','item_id','item_cnt_day']]

In [ ]:
sales_train.groupby('shop_id')['item_id'].count()

In [ ]:
df=sales_train[['item_cnt_day','shop_id','item_id']].groupby(['shop_id','item_id']).count()
df.reset_index()

In [ ]:
df=sales_train[['item_cnt_day','shop_id','item_id']].groupby(['shop_id','item_id']).sum()
df.reset_index()

In [ ]:
df.max()

In [ ]:
sales_train[sales_train['shop_id']==28]

In [ ]:
df.plot(figsize=(20,5))

**Feature engineering

In [ ]:
dataset=sales_train.pivot_table(index=['shop_id','item_id'],values=['item_cnt_day'],columns=['date_block_num'],fill_value=0,aggfunc='sum')

In [ ]:
dataset.reset_index(inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset=pd.merge(test,dataset,on=['item_id','shop_id'],how='left')

In [ ]:
dataset.fillna(0,inplace=True)
dataset.head()

In [ ]:
dataset.drop(['shop_id','item_id','ID'],inplace=True,axis=1)
dataset.head()

**Machine Learning

In [ ]:
X_train=np.expand_dims(dataset.values[:,:-1],axis=2)

y_train=dataset.values[:,-1:]

X_test=np.expand_dims(dataset.values[:,1:],axis=2)

print(X_train.shape,y_train.shape,X_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [ ]:
model=Sequential()
model.add(LSTM(units=64,input_shape=(33,1)))
model.add(Dropout(0.4))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam',metrics=['mean_squared_error'])
model.summary()

In [ ]:
model.fit(X_train,y_train,batch_size=4096,epochs=10)

In [ ]:
sales_submission=model.predict(X_test)
sales_submission=sales_submission.clip(0.20)
submission=pd.DataFrame({'ID':test['ID'],'item_cnt_month':sales_submission.ravel()})
submission.to_csv('sales_sub.csv',index=False)